In [80]:
from tf_keras_model import get_particle_net_lite,get_particle_net
from tensorflow import keras
import logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s: %(message)s')
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
dataset = np.load('./data/QG_jets.npz',allow_pickle=True)

In [11]:
list(dataset.keys())

['X', 'y']

In [84]:
x = dataset['X']
y = dataset['y']

In [85]:
print(x.shape)
print(y.shape)

(100000, 139, 4)
(100000,)


In [86]:
# one hot encoding labels is needed else the following error 
# ValueError: Shapes (None, 1) and (None, 2) are incompatible
y = tf.keras.utils.to_categorical(y)
print(y.shape)

(100000, 2)


In [88]:


# From paper
# The first EdgeConv block uses the spatial coordinates
# of the particles in the pseudorapidity-azimuth space to
# compute the distances, while the subsequent blocks use
# the learned feature vectors as coordinates.

# from dataset
# features of each particle are its pt, rapidity, azimuthal angle, and pdgid

# Shapes of each input (`points`, `features`, `mask`)


In [89]:
x, y = shuffle(x, y, random_state=0)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test , test_size=0.5, random_state=42)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(x_val.shape, y_val.shape)

(70000, 139, 4) (70000, 2)
(15000, 139, 4) (15000, 2)
(15000, 139, 4) (15000, 2)


In [90]:
# mask
# find the maximum length vector in for each sample in 10000 samples with maximum number of non-zero values
mask_train = np.sum(x_train, axis=2) 
print(mask_train.shape)
print(mask_train[0])

# make the array binary
mask_train = np.array(mask_train !=0 , np.float32)
print(mask_train[0])

# reshape mask for a third axis
mask_train = mask_train.reshape(x_train.shape[0], x_train.shape[1], 1)
print(mask_train.shape)

(70000, 139)
[   29.20243269    27.50561143    30.26089051   220.41819997
    27.95042087    28.15969933   329.0687446     27.57628735
  -206.06003465   218.08468374    26.41602355    27.02195013
    31.79622486    26.59246689    26.88924931  -202.09822736
    26.71899363    31.23462352  -310.42681747    27.70451205
    30.7169403   -198.56554041  2233.77838649    36.42720615
    29.23621805  -187.38676832    33.46071738   268.17256202
    32.3521241  -2054.01856237   118.7019814     56.63953271
   151.53934524    57.07618763     0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.

In [91]:
# mask
# find the maximum length vector in for each sample in 10000 samples with maximum number of non-zero values
mask_val = np.sum(x_val, axis=2) 

# make the array binary
mask_val = np.array(mask_val !=0 , np.float32)

# reshape mask for a third axis
mask_val = mask_val.reshape(x_val.shape[0], x_val.shape[1], 1)
print(mask_val.shape)

(15000, 139, 1)


In [92]:
# mask
# find the maximum length vector in for each sample in 10000 samples with maximum number of non-zero values
mask_test = np.sum(x_test, axis=2) 

# make the array binary
mask_test = np.array(mask_test !=0 , np.float32)

# reshape mask for a third axis
mask_test = mask_val.reshape(x_test.shape[0], x_test.shape[1], 1)
print(mask_test.shape)

(15000, 139, 1)


In [93]:
train_dataset = {
    'points': x_train[:,:,1:3],
    'features': x_train,
    'mask': mask_train
}

test_dataset = {
    'points': x_test[:,:,1:3],
    'features': x_test,
    'mask': mask_test
}

val_dataset = {
    'points': x_val[:,:,1:3],
    'features': x_val,
    'mask': mask_val
}

In [94]:
shapes = {
    'points': x_train[:,:,1:3].shape[1:],
    'features': x_train.shape[1:],
    'mask': mask_train.shape[1:]
}
shapes

{'points': (139, 2), 'features': (139, 4), 'mask': (139, 1)}

In [95]:
# particle net lite
num_classes = 2
model = get_particle_net_lite(num_classes, shapes)

In [96]:
# Training parameters
batch_size = 1024
epochs = 3

In [97]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 10:
        lr *= 0.1
    elif epoch > 20:
        lr *= 0.01
    logging.info('Learning rate: %f'%lr)
    return lr

In [98]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()

[2022-04-17 14:34:59,212] INFO: Learning rate: 0.001000


Model: "ParticleNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 mask (InputLayer)              [(None, 139, 1)]     0           []                               
                                                                                                  
 tf.math.not_equal_3 (TFOpLambd  (None, 139, 1)      0           ['mask[0][0]']                   
 a)                                                                                               
                                                                                                  
 tf.cast_6 (TFOpLambda)         (None, 139, 1)       0           ['tf.math.not_equal_3[0][0]']    
                                                                                                  
 tf.math.equal_3 (TFOpLambda)   (None, 139, 1)       0           ['tf.cast_6[0][0]']    

 tf.__operators__.getitem_8 (Sl  (None, 139, 7)      0           ['tf.math.top_k_4[0][1]']        
 icingOpLambda)                                                                                   
                                                                                                  
 tf.tile_8 (TFOpLambda)         (None, 139, 7, 1)    0           ['tf.reshape_4[0][0]']           
                                                                                                  
 tf.expand_dims_16 (TFOpLambda)  (None, 139, 7, 1)   0           ['tf.__operators__.getitem_8[0][0
                                                                 ]']                              
                                                                                                  
 tf.expand_dims_17 (TFOpLambda)  (None, 139, 1, 4)   0           ['tf.compat.v1.squeeze_7[0][0]'] 
                                                                                                  
 tf.concat

                                                                 ']                               
                                                                                                  
 tf.math.multiply_24 (TFOpLambd  (None, 139, 32)     0           ['tf.math.add_6[0][0]',          
 a)                                                               'tf.math.add_6[0][0]']          
                                                                                                  
 tf.math.reduce_sum_12 (TFOpLam  (None, 139, 1)      0           ['tf.math.multiply_23[0][0]']    
 bda)                                                                                             
                                                                                                  
 tf.math.multiply_25 (TFOpLambd  (None, 139, 139)    0           ['tf.linalg.matmul_5[0][0]']     
 a)                                                                                               
          

                                                                                                  
 ParticleNet_EdgeConv1_act1 (Ac  (None, 139, 7, 64)  0           ['ParticleNet_EdgeConv1_bn1[0][0]
 tivation)                                                       ']                               
                                                                                                  
 tf.expand_dims_21 (TFOpLambda)  (None, 139, 1, 32)  0           ['ParticleNet_EdgeConv0_sc_act[0]
                                                                 [0]']                            
                                                                                                  
 ParticleNet_EdgeConv1_conv2 (C  (None, 139, 7, 64)  4096        ['ParticleNet_EdgeConv1_act1[0][0
 onv2D)                                                          ]']                              
                                                                                                  
 ParticleN

In [99]:
# Prepare model model saving directory.

save_dir = 'model_checkpoints'
model_name = '%s_model.{epoch:03d}.h5' % 'particle_net_lite'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = keras.callbacks.ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)
progress_bar = keras.callbacks.ProgbarLogger()
callbacks = [checkpoint, lr_scheduler, progress_bar]

In [100]:
history = model.fit(train_dataset, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(val_dataset, y_val),
          shuffle=True,
          callbacks=callbacks)

2022-04-17 14:35:02.841615: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 155680000 exceeds 10% of free system memory.
[2022-04-17 14:35:03,254] INFO: Learning rate: 0.001000


Epoch 1/3


2022-04-17 14:35:09.874349: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 127533056 exceeds 10% of free system memory.


      0/Unknown - 56s 0s/sample - loss: 0.7741 - accuracy: 0.5068


KeyboardInterrupt

